<a href="https://colab.research.google.com/github/d61h6k4/notebooks/blob/master/SLAM/Sheet1/Ex2_OdometryModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Course:  [Robot Mapping](http://ais.informatik.uni-freiburg.de/teaching/ws13/mapping/) 

[Sheet1](http://ais.informatik.uni-freiburg.de/teaching/ws13/mapping/exercise/robotMappingSheet1.pdf) Exercise 2: Implementing an odometry model

In this course was used octave as a programmin language, I will use python, so I have to implement all helper tools to.

I will use dat file from data folder of Sheet1 octave code.

In [0]:
%matplotlib inline

import dataclasses
import numpy as np
import os
import matplotlib

from typing import Dict, List, TypeVar

In [0]:
matplotlib.pyplot.style.use('fivethirtyeight')

Python version of tools/read_world.m

In [0]:
LandmarkId = TypeVar(int)

@dataclasses.dataclass
class CartesianPoint:
    x: float
    y: float

In [0]:
def read_world(filename: os.PathLike) -> Dict[LandmarkId, CartesianPoint]:
    """
    The original doc of function `read_world`:
    % Reads the world definition and returns a structure of landmarks.
    %
    % filename: path of the file to load
    % landmarks: structure containing the parsed information
    %
    % Each landmark contains the following information:
    % - id : id of the landmark
    % - x  : x-coordinate
    % - y  : y-coordinate
    %
    % Examples:
    % - Obtain x-coordinate of the 5-th landmark
    %   landmarks(5).x
    in python we will represent it as Dict[LandmarkId, CartesianPoint]
    """
    landmarks = {}
    with open(filename, 'r') as src:
        for line in src:
            lid, x, y = line.strip().split(' ')
            landmarks[int(lid)] = CartesianPoint(float(x), float(y))
    return landmarks

Python version of tools/read_data.m

In [0]:
@dataclasses.dataclass
class Odometry:
    r1: float
    t: float
    r2: float


SensorId = TypeVar(int)


@dataclasses.dataclass
class Sensor:
    range: float
    bearing: float


class DataPoint(object):
    def __init__(self):
        self.__empty = True

        self.__odometry: Optional[Odometry] = None
        self.__sensor: Dict[SensorId, Sensor] = {}

    def __eq__(self) -> bool:
        return self.__empty

    def set_odometry(self, odometry: Odometry):
        self.__empty = False
        self.__odometry = odometry

    def add_sensor(self, sensor_id: SensorId, sensor: Sensor):
        self.__empty = False
        pass

    def clear(self):
        self.__empty = True
        self.__odometry = None
        self.__sensor = {}

    @property
    def odometry(self) -> Odometry:
        return self.__odometry

    @property
    def sensor(self) -> Dict[SensorId, Sensor]:
        return self.__sensor

In [0]:
def read_data(filename: os.PathLike) -> List[DataPoint]:
    """
    % Reads the odometry and sensor readings from a file.
    %
    % filename: path to the file to parse
    % data: structure containing the parsed information
    %
    % The data is returned in a structure where the u_t and z_t are stored
    % within a single entry. A z_t can contain observations of multiple
    % landmarks.
    %
    % Usage:
    % - access the readings for timestep i:
    %   data.timestep(i)
    %   this returns a structure containing the odometry reading and all
    %   landmark obsevations, which can be accessed as follows
    % - odometry reading at timestep i:
    %   data.timestep(i).odometry
    % - senor reading at timestep i:
    %   data.timestep(i).sensor
    %
    % Odometry readings have the following fields:
    % - r1 : rotation 1
    % - t  : translation
    % - r2 : rotation 2
    % which correspond to the identically labeled variables in the motion
    % mode.
    %
    % Sensor readings can again be indexed and each of the entris has the
    % following fields:
    % - id      : id of the observed landmark
    % - range   : measured range to the landmark
    % - bearing : measured angle to the landmark (you can ignore this)
    %
    % Examples:
    % - Translational component of the odometry reading at timestep 10
    %   data.timestep(10).odometry.t
    % - Measured range to the second landmark observed at timestep 4
    %   data.timestep(4).sensor(2).range
    """
    data = []
    data_point = DataPoint()

    with open(filename, 'r') as src:
        for line in src:
            row = line.strip().split(' ')
            if row[0] == "ODOMETRY":
                if data_point:
                    data.append(data_point)
                    data_point.clear()
                data_point.set_odometry(Odometry(float(row[1]), float(row[2]), float(row[3])))
            elif row[0] == "SENSOR":
                data_point.add_sensor(int(row[1]), Sensor(float(row[2]), float(row[3])))
    return data

In [0]:
def plot_state(x: np.array, landmarks: Dict[LandmarkId, CartesianPoint], t: int, sensors: Dict[SensorId, Sensor]):
    """
    % Visualizes the robot in the map.
    %
    % The resulting plot displays the following information:
    % - the landmarks in the map (black +'s)
    % - current robot pose (red)
    % - observations made at this time step (line between robot and landmark)

    clf
    hold on
    grid("on")
    L = struct2cell(landmarks);
    figure(1, "visible", "off");
    plot(cell2mat(L(2,:)), cell2mat(L(3,:)), 'k+', 'markersize', 10, 'linewidth', 5);

    for(i=1:size(z,2))
        id = z(i).id;
	mX = landmarks(id).x;
	mY = landmarks(id).y;
    	line([mu(1), mX],[mu(2), mY], 'color', 'b', 'linewidth', 1);
    endfor

    drawrobot(mu(1:3), 'r', 3, 0.3, 0.3);
    xlim([-2, 12])
    ylim([-2, 12])
    filename = sprintf('../plots/odom_%03d.png', timestep);
    print(filename, '-dpng');
    hold off
    end
    """
    landmark_coords = np.array([[coord.x, coord.y] for coord in landmarks.values()])

    fig, ax = matplotlib.pyplot.subplots()
    ax.plot(landmark_coords[:,0], landmark_coords[:,1], 'k+', markersize=10, linewidth=5)

    for landmark_coord in landmark_coords:
        ax.add_line(matplotlib.lines.Line2D([x[0], landmark_coord[0]], [x[1], landmark_coord[1]], color='b', linewidth=1))

    ax.plot([x[0]], [x[1]], marker='o', linewidth=0, markerfacecolor='w', markeredgecolor='r')

    ax.grid(True)
    ax.set_xlim([-2, 12])
    ax.set_ylim([-2, 12])

    fig.savefig(f"plots/odom_{t}.png")

    fig.set_visible(False)
    return fig

In [0]:
def motion_command(x: np.array, odometry: Odometry) -> np.array:
    return x

Here is implementaiton of main.m

In [20]:
landmarks = read_world('world.dat')
data = read_data('sensor_data.dat')

x = np.zeros(3)
for t, data_point in enumerate(data):
    x = motion_command(x, data_point.odometry)
    _ = plot_state(x, landmarks, t, data_point.sensor)
    print(f"Current robot pose: x={x}")
    break

print(f"Finar robot pose is x={x}")

ValueError: ignored

ValueError: ignored

<Figure size 432x288 with 1 Axes>

In [11]:
!ls plots

odom_0.png
